In [ ]:
# ЯЧЕЙКА 1 — Импорты
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Убедись, что папки model_raw... и model_finetuned_final лежат в той же директории

In [ ]:
# ЯЧЕЙКА 2 — Загружаем обе модели одной строкой (всё!)
# ЯЧЕЙКА — Загружаем ВСЕ 4 модели: Bi и Cross из raw + из fine-tuned
from sentence_transformers import SentenceTransformer, CrossEncoder

print("Загружаем модели...")

# 1. Модели ИЗ КОРОБКИ (raw)
bi_raw        = SentenceTransformer("model_raw_RuModernBERT")           # Bi-Encoder из коробки
cross_raw     = CrossEncoder("cross_encoder_raw")                      # Cross-Encoder из коробки (сохранённый ранее)

# 2. Модели ПОСЛЕ ДООБУЧЕНИЯ (fine-tuned)
bi_finetuned  = SentenceTransformer("bi_finetuned_final")             # Дообученный Bi-Encoder
cross_finetuned = CrossEncoder("cross_finetuned_final")               # Дообученный Cross-Encoder

print("Все 4 модели успешно загружены!\n")

# Красивый вывод размерностей
print(f"{'Модель':<35} {'Тип':<12} {'Размерность':<12} {'Путь'}")
print("-" * 80)
print(f"{'bi_raw':<35} {'Bi-Encoder':<12} {bi_raw.get_sentence_embedding_dimension():<12} {'model_raw_RuModernBERT'}")
print(f"{'cross_raw':<35} {'Cross-Encoder':<12} {'—':<12} {'cross_encoder_raw'}")
print(f"{'bi_finetuned':<35} {'Bi-Encoder':<12} {bi_finetuned.get_sentence_embedding_dimension():<12} {'bi_finetuned_final'}")
print(f"{'cross_finetuned':<35} {'Cross-Encoder':<12} {'—':<12} {'cross_finetuned_final'}")

# Проверка, что всё работает
print("\nПроверка: считаем эмбеддинг от bi_finetuned...")
test_emb = bi_finetuned.encode("тест", normalize_embeddings=True)
print(f"Всё ок! Размер эмбеддинга: {test_emb.shape}")

In [ ]:
# ЯЧЕЙКА 3 — Загружаем тестовый датасет и корпус постов
df_test = pd.read_csv("test_pairs.csv")        # query, post, score
df_corpus = pd.read_csv("all_posts.csv")       # post_text, channel_name

# Уникальные посты и их каналы
post_to_channel = dict(zip(df_corpus['post_text'], df_corpus['channel_name']))
posts = df_corpus['post_text'].astype(str).unique().tolist()

print(f"Тестовых пар: {len(df_test)}")
print(f"Постов в корпусе: {len(posts)}")

In [ ]:
# ЯЧЕЙКА 4 — Функция подсчёта всех метрик (универсальная)
def evaluate_model(model, name="Модель"):
    print(f"\n=== Оценка: {name} ===")
    
    # 1. Spearman
    q_emb = model.encode(df_test['query'].tolist(), normalize_embeddings=True, batch_size=64, show_progress_bar=False)
    p_emb = model.encode(df_test['post'].tolist(),  normalize_embeddings=True, batch_size=64, show_progress_bar=False)
    cos_scores = [util.cos_sim(q, p).item() for q, p in zip(q_emb, p_emb)]
    spearman = spearmanr(cos_scores, df_test['score']).correlation
    
    # 2–4. Precision@10, Recall@50, MRR (по каналам!)
    prec10, recall50, mrr = 0.0, 0.0, 0.0
    queries = df_test['query'].unique()
    
    for query in tqdm(queries, desc="Оценка запросов"):
        # Эталон: все каналы с score >= 0.7 для этого запроса
        relevant_posts = df_test[(df_test['query'] == query) & (df_test['score'] >= 0.7)]['post'].tolist()
        relevant_channels = set(post_to_channel.get(p, "unknown") for p in relevant_posts if p in post_to_channel)
        
        # Ранжируем весь корпус
        q_vec = model.encode([query], normalize_embeddings=True)[0]
        p_vecs = model.encode(posts, normalize_embeddings=True, batch_size=64, show_progress_bar=False)
        scores = util.cos_sim(q_vec, p_vecs)[0].cpu().numpy()
        
        top_indices = np.argsort(scores)[::-1]
        ranked_channels = []
        channel_scores = {}
        
        for idx in top_indices:
            post_text = posts[idx]
            channel = post_to_channel.get(post_text, "unknown")
            channel_scores[channel] = max(channel_scores.get(channel, 0), scores[idx])
            ranked_channels.append((channel, channel_scores[channel]))
        
        # Топ-10 и топ-50 каналов
        ranked_channels = sorted(channel_scores.items(), key=lambda x: x[1], reverse=True)
        top10 = [ch for ch, _ in ranked_channels[:10]]
        top50 = [ch for ch, _ in ranked_channels[:50]]
        
        # Метрики
        hits10 = len(set(top10) & relevant_channels)
        prec10 += hits10 / 10
        
        hits50 = len(set(top50) & relevant_channels)
        recall50 += hits50 / max(1, len(relevant_channels))
        
        for rank, (ch, _) in enumerate(ranked_channels):
            if ch in relevant_channels:
                mrr += 1.0 / (rank + 1)
                break
    
    n = len(queries)
    result = {
        "Spearman": round(spearman, 4),
        "Precision@10": round(prec10 / n, 4),
        "Recall@50":    round(recall50 / n, 4),
        "MRR":          round(mrr / n, 4),
    }
    
    for k, v in result.items():
        print(f"{k}: {v}")
    
    return result

# Запускаем сравнение
raw_results       = evaluate_model(model_raw,       "Из коробки")
finetuned_results = evaluate_model(model_finetuned, "После дообучения")

In [ ]:
# ЯЧЕЙКА 5 — Красивая таблица и график
import pandas as pd
import matplotlib.pyplot as plt

from recommend import recommend_channels

# Пример: сравнение двух моделей на одном запросе
query = "Детская коляска 3 в 1 с автокреслом"

print("=== Из коробки ===")
print(recommend_channels(query, model_raw, cross_encoder, index, df, top_k_channels=5))

print("\n=== После дообучения ===")
print(recommend_channels(query, model_finetuned, cross_encoder, index, df, top_k_channels=5))